In [ ]:
#Build a Feed Forward Neural Network for any problems with keras tuner.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from kerastuner.tuners import RandomSearch

C:\Users\ankit\AppData\Local\Temp\ipykernel_19108\3648249868.py:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
# Load and preprocess the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

4422102/4422102 [==============================] - 0s 0us/step


In [3]:
# Define the model-building function for Keras Tuner
def build_model(hp):
    model = keras.Sequential()

    # Flatten the input layer
    model.add(layers.Flatten(input_shape=(28, 28, 1)))

    # Tune the number of units in the first dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units, activation='relu'))

    # Tune the number of hidden layers
    hp_layers = hp.Int('num_layers', min_value=1, max_value=3)
    for _ in range(hp_layers):
        model.add(layers.Dense(units=hp_units, activation='relu'))

    # Output layer with 10 units (classes) and softmax activation
    model.add(layers.Dense(10, activation='softmax'))

    # Tune the learning rate and optimizer choice
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    optimizer_choice = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])

    if optimizer_choice == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate)
    elif optimizer_choice == 'sgd':
        optimizer = keras.optimizers.SGD(learning_rate=hp_learning_rate)
    else:
        optimizer = keras.optimizers.RMSprop(learning_rate=hp_learning_rate)

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [4]:
# Instantiate the RandomSearch tuner and perform the hyperparameter search
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Number of different hyperparameter combinations to try
    directory='keras_tuner_results',  # Directory to save the results
    project_name='fashion_mnist_tuning'
)

# Search for the best hyperparameter configuration
tuner.search(train_images, train_labels, epochs=5, validation_split=0.1)

# Get the best model and evaluate it on the test set
best_model = tuner.get_best_models(num_models=1)[0]
test_loss, test_acc = best_model.evaluate(test_images, test_labels)

print(f'\nTest accuracy with the best model: {test_acc}')

Trial 2 Complete [00h 00m 25s]
val_accuracy: 0.6439999938011169

Best val_accuracy So Far: 0.6608333587646484
Total elapsed time: 00h 01m 11s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
32                |480               |units
2                 |2                 |num_layers
0.0001            |0.0001            |learning_rate
adam              |sgd               |optimizer

Epoch 1/5
1688/1688 [==============================] - 5s 2ms/step - loss: 0.9077 - accuracy: 0.6956 - val_loss: 0.5841 - val_accuracy: 0.7975
Epoch 2/5
1688/1688 [==============================] - 3s 2ms/step - loss: 0.5322 - accuracy: 0.8171 - val_loss: 0.4934 - val_accuracy: 0.8320
Epoch 3/5
1688/1688 [==============================] - 4s 2ms/step - loss: 0.4763 - accuracy: 0.8368 - val_loss: 0.4670 - val_accuracy: 0.8372
Epoch 4/5
1688/1688 [==============================] - 4s 2ms/step - loss: 0.4463 - accuracy: 0.8465 - val_loss: 0.4365 - val_accuracy: 0.8477
Epoch 5/5
14

KeyboardInterrupt: 